In [2]:
import tensorflow as tf
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.test.is_built_with_cuda()

True

# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential,load_model
from keras.layers import Input, Embedding
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import LSTM
import keras.metrics as metrics
import itertools
from tensorflow.python.keras.utils.data_utils import Sequence
from decimal import Decimal
from keras import backend as K
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense

# Data Fetching

In [5]:
A1=np.empty((0,5),dtype='float32')
U1=np.empty((0,7),dtype='float32')
node=['150','149','147','144','142','140','136','61']
mon=['Apr','Mar','Aug','Jun','Jul','Sep','May','Oct']
for j in node:
  for i in mon:
    inp= pd.read_csv('data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[1,2,3,15,16])
    out= pd.read_csv('data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[5,6,7,8,17,18,19])
    
    inp=np.array(inp,dtype='float32')
    out=np.array(out,dtype='float32')
    
    A1=np.append(A1, inp, axis=0)
    U1=np.append(U1, out, axis=0)

print(A1)
print(U1)


[[1.50000e+02 1.90401e+05 7.25000e+02 2.75500e+01 8.03900e+01]
 [1.50000e+02 1.90401e+05 8.25000e+02 2.75600e+01 8.03300e+01]
 [1.50000e+02 1.90401e+05 9.25000e+02 2.75800e+01 8.02400e+01]
 ...
 [6.10000e+01 1.91020e+05 1.94532e+05 2.93700e+01 7.52100e+01]
 [6.10000e+01 1.91020e+05 1.94632e+05 2.93500e+01 7.52700e+01]
 [6.10000e+01 1.91020e+05 1.94732e+05 2.93400e+01 7.53000e+01]]
[[ 28.     3.   -52.   ...  16.97  19.63  20.06]
 [ 28.    15.   -53.   ...  16.63  19.57  23.06]
 [ 31.    16.   -55.   ...  17.24  19.98  20.24]
 ...
 [ 76.    12.   -76.   ...   3.47   3.95   4.35]
 [ 75.    13.   -76.   ...   3.88   4.33   4.42]
 [ 76.    12.   -75.   ...   3.46   4.07   4.28]]


# Min Max Scaler

In [6]:
from sklearn.preprocessing import MinMaxScaler
import warnings
scaler_obj=MinMaxScaler()
X1=scaler_obj.fit_transform(A1)
Y1=scaler_obj.fit_transform(U1)

warnings.filterwarnings(action='ignore', category=UserWarning)

X1=X1[:,np.newaxis,:]
Y1=Y1[:,np.newaxis,:]

In [7]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Model

In [11]:
model1 = Sequential()
model1.add(keras.Input(shape=(1,5)))
model1.add(tf.keras.layers.LSTM(7,activation="tanh",use_bias=True,kernel_initializer="glorot_uniform",bias_initializer="zeros"))
model1.add(Dense(7))
model1.add(keras.layers.BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,center=True,scale=True,
                                beta_initializer="zeros",gamma_initializer="ones",
                                moving_mean_initializer="zeros",moving_variance_initializer="ones",trainable=True))
model1.add(keras.layers.ReLU())
model1.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy',metrics=['accuracy','mse','mae',rmse])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 7)                 364       
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 56        
_________________________________________________________________
batch_normalization_1 (Batch (None, 7)                 28        
_________________________________________________________________
re_lu_1 (ReLU)               (None, 7)                 0         
Total params: 448
Trainable params: 434
Non-trainable params: 14
_________________________________________________________________


In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.25, random_state=42)

model_fit8 = model1.fit(x_train,y_train,batch_size=2048,epochs=50, validation_split=0.1)

Epoch 1/50
571/571 [==============================] - 252s 439ms/step - loss: 0.6914 - accuracy: 0.0134 - mse: 0.0722 - mae: 0.1448 - rmse: 0.2206 - val_loss: 0.2356 - val_accuracy: 0.0060 - val_mse: 0.0201 - val_mae: 0.0750 - val_rmse: 0.1165
Epoch 2/50
571/571 [==============================] - 252s 441ms/step - loss: 0.6704 - accuracy: 0.0099 - mse: 0.0658 - mae: 0.1312 - rmse: 0.2070 - val_loss: 0.6566 - val_accuracy: 0.0072 - val_mse: 0.0621 - val_mae: 0.1221 - val_rmse: 0.1962
Epoch 3/50
571/571 [==============================] - 250s 438ms/step - loss: 0.6523 - accuracy: 0.0066 - mse: 0.0613 - mae: 0.1185 - rmse: 0.1950 - val_loss: 0.6402 - val_accuracy: 0.0059 - val_mse: 0.0587 - val_mae: 0.1102 - val_rmse: 0.1850
Epoch 4/50
571/571 [==============================] - 250s 437ms/step - loss: 0.6366 - accuracy: 0.0042 - mse: 0.0582 - mae: 0.1065 - rmse: 0.1846 - val_loss: 0.6267 - val_accuracy: 0.0038 - val_mse: 0.0563 - val_mae: 0.0994 - val_rmse: 0.1760
Epoch 5/50
571/571 [====

571/571 [==============================] - 248s 435ms/step - loss: 0.5789 - accuracy: 0.0162 - mse: 0.0534 - mae: 0.0538 - rmse: 0.1246 - val_loss: 0.5772 - val_accuracy: 0.0171 - val_mse: 0.0533 - val_mae: 0.0534 - val_rmse: 0.1240
Epoch 35/50
571/571 [==============================] - 248s 434ms/step - loss: 0.5785 - accuracy: 0.0161 - mse: 0.0536 - mae: 0.0535 - rmse: 0.1244 - val_loss: 0.5769 - val_accuracy: 0.0158 - val_mse: 0.0534 - val_mae: 0.0533 - val_rmse: 0.1239
Epoch 36/50
571/571 [==============================] - 248s 434ms/step - loss: 0.5782 - accuracy: 0.0161 - mse: 0.0537 - mae: 0.0532 - rmse: 0.1242 - val_loss: 0.5766 - val_accuracy: 0.0156 - val_mse: 0.0538 - val_mae: 0.0530 - val_rmse: 0.1239
Epoch 37/50
571/571 [==============================] - 248s 434ms/step - loss: 0.5780 - accuracy: 0.0165 - mse: 0.0539 - mae: 0.0529 - rmse: 0.1239 - val_loss: 0.5765 - val_accuracy: 0.0167 - val_mse: 0.0537 - val_mae: 0.0524 - val_rmse: 0.1232
Epoch 38/50
571/571 [===========

# Saving Model as File

In [13]:
model_json = model1.to_json()
with open("lstm.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model1.save_weights("lstm.h5")
print("Saved model to disk")

Saved model to disk


In [14]:
from keras.models import model_from_json
json_file = open('lstm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("lstm.h5")
print("Loaded model from disk")
loaded_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse',metrics=['accuracy','mse','mae',rmse,coeff_determination])
print(loaded_model.evaluate(x_train, y_train, verbose=0))

Loaded model from disk
[0.05459023267030716, 0.02124212495982647, 0.05459020659327507, 0.05129491165280342, 0.12296319007873535, -2923.8349609375]


# Error Analysis

In [15]:
_, train_acc = model1.evaluate(x_train, y_train, verbose=0)
_, test_acc = model1.evaluate(x_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

# summarize history for loss
plt.plot(model_fit8.history['loss'])
plt.plot(model_fit8.history['val_loss'])
plt.title('Model Loss',fontweight ='bold',fontsize = 15)
plt.ylabel('Loss',fontweight ='bold',fontsize = 15)
plt.xlabel('Epoch',fontweight ='bold',fontsize = 15)
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# summarize history for accuracy
plt.plot(model_fit8.history['accuracy'])
plt.plot(model_fit8.history['val_accuracy'])
plt.title('Model accuracy',fontweight ='bold',fontsize = 15)
plt.ylabel('Accuracy',fontweight ='bold',fontsize = 15)
plt.xlabel('Epoch',fontweight ='bold',fontsize = 15)
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

ValueError: too many values to unpack (expected 2)